In [1]:
import numpy as np
import h5py
import glob
import os
import sys
import re
import matplotlib.pyplot as plt
%matplotlib inline
from os.path import expanduser
from itertools import zip_longest
home = expanduser("~")
host = os.getenv('HOSTNAME')
print(host)

None


In [2]:
if host is not None:
    if 'bebop' in host:
        input_dir = '/lcrc/project/galsampler/Catalog_5000/OR_5000/'
        size_dir = '/lcrc/project/galsampler/Validation_Data/Sizes'
else:
    #input_dir = '/lus/eagle/projects/LastJourney/kovacs/Catalog_5000/OR_5000/'
    #size_dir = '/lus/eagle/projects/LastJourney/kovacs/Validation_Data/Sizes'
    input_dir = '/Users/kovacs/Catalog_5000/OR_5000/roman_rubin_2023_v1.1.3'
    size_dir = '/Users/kovacs/Validation_Data/Sizes'
fname = 'roman_rubin_2023_z_*_cutout_{}.hdf5'
from lsstdesc_diffsky.validation.get_catalog_data import get_fhlist
from lsstdesc_diffsky.validation.get_catalog_data import get_colnames
from lsstdesc_diffsky.write_mock_to_disk import get_astropy_table
from lsstdesc_diffsky.size_modeling import zhang_yang17

AttributeError: jnp.trapz is deprecated; use jnp.trapezoid instead.

In [ ]:
data_dict = {'martorano_24': {'filename': 'martorano_2024_table1_Mstar_log10Re_{}.txt',
                              'samples': ['all', 'star_forming', 'quiescent']
                             }
            }
def read_size_fits(author, galaxy_sample):
    